# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [21]:
df_train = pd.read_csv('train_prob.csv', index_col=['id'])
df_test = pd.read_csv('test_prob.csv', index_col=['id'])

In [22]:
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)
df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)

In [23]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
imp = IterativeImputer(
    estimator=LinearRegression(fit_intercept=True),
    random_state=123
)
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)
df_train[X_imp].isna().sum().to_frame().T

,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_17
0,0,0,0,0,0,0,0,0


In [24]:
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)
df_test[X_imp].isna().sum().to_frame().T

,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_17
0,0,0,0,0,0,0,0,0


In [25]:
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))
df_train[X_mean].isna().sum().to_frame().T

,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16
0,0,0,0,0,0,0,0


In [26]:
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))
df_test[X_mean].isna().sum().to_frame().T

,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16
0,0,0,0,0,0,0,0


In [33]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())

- loaing_log: loading 로그 변환

- LR + SFS: \['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1'\]

- Linear Discriminant Analysis: 변환기, 분류기

- LR + PCA(n_components=7)

- RandomForestClassifier {'max_depth': 7, 'min_samples_split': 512, 'n_estimators': 15}

# Kaggle형 풀이 단계

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

Step 4: 모델 개선 작업을 합니다.

In [27]:
from sklearn.model_selection import GroupKFold
gcv = GroupKFold(4)

In [29]:
X_all = df_test.columns.tolist()

In [19]:
for train_idx, valid_idx in gcv.split(df_train[X_all], df_train['failure'], groups=df_train['product_code']):
    print(df_train.iloc[train_idx]['product_code'].unique(), df_train.iloc[valid_idx]['product_code'].unique())

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GroupKFold
gcv = GroupKFold(4)

ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
cross_val_score(clf_lr, df_train[X_all], df_train['failure'], scoring='roc_auc', cv=gcv, groups=df_train['product_code'])

array([0.58822089, 0.58492694, 0.58894173, 0.59538985])

In [37]:
from sklearn.model_selection import cross_validate
cross_validate(
    clf_lr, df_train[X_all], df_train['failure'], scoring='roc_auc', cv=gcv, groups=df_train['product_code'],
    return_train_score=True
)

{'fit_time': array([0.03513026, 0.0369544 , 0.03010917, 0.02981114]),
 'score_time': array([0.00530553, 0.        , 0.01397347, 0.00977278]),
 'test_score': array([0.58822089, 0.58492694, 0.58894173, 0.59538985]),
 'train_score': array([0.59262299, 0.59350682, 0.59192443, 0.58956962])}